In [1]:
"""
=================================================================
PHASE 4 & 5: EXPLORATORY DATA ANALYSIS (EDA) & KPIs
=================================================================
This notebook answers mandatory analysis questions and calculates all KPIs
=================================================================
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 7)

# Load cleaned data
df = pd.read_csv('../data/processed/superstore_cleaned.csv', encoding='latin-1')

# Convert date columns
for col in df.columns:
    if 'date' in col.lower():
        df[col] = pd.to_datetime(df[col])

print(f"✅ Data loaded: {df.shape}")
print(f"📅 Date range: {df['Order Date'].min()} to {df['Order Date'].max()}")
print("\n" + "="*80)
print("PHASE 4 & 5: EDA & KPI CALCULATIONS")
print("="*80)

# ============================================================================
# ANALYSIS 1: MONTHLY & YEARLY REVENUE TRENDS
# ============================================================================
print("\n\n1️⃣ ANALYSIS: MONTHLY & YEARLY REVENUE TRENDS")
print("-" * 80)

monthly_revenue = df.groupby(df['Order Date'].dt.to_period('M')).agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Order ID': 'count'
}).reset_index()
monthly_revenue.columns = ['Month', 'Revenue', 'Profit', 'Orders']
monthly_revenue['Month'] = monthly_revenue['Month'].astype(str)

yearly_revenue = df.groupby(df['Order Date'].dt.year).agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Order ID': 'count'
}).reset_index()
yearly_revenue.columns = ['Year', 'Revenue', 'Profit', 'Orders']

print(f"Total Months: {len(monthly_revenue)}")
print(f"\nYearly Revenue Summary:")
print(yearly_revenue)

# Visualization
fig, axes = plt.subplots(2, 1, figsize=(14, 8))
axes[0].plot(monthly_revenue.index, monthly_revenue['Revenue'], marker='o', label='Revenue', linewidth=2)
axes[0].set_title('Monthly Revenue Trends', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Revenue ($)')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

axes[1].plot(monthly_revenue.index, monthly_revenue['Orders'], marker='s', label='Orders', color='orange', linewidth=2)
axes[1].set_title('Monthly Order Volume', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Number of Orders')
axes[1].set_xlabel('Month')
axes[1].grid(True, alpha=0.3)
axes[1].legend()
plt.tight_layout()
plt.savefig('../reports/01_revenue_trends.png', dpi=300, bbox_inches='tight')
print("✅ Saved: 01_revenue_trends.png")
plt.close()

# ============================================================================
# ANALYSIS 2: TOP 10 PRODUCTS BY REVENUE & PROFIT
# ============================================================================
print("\n\n2️⃣ ANALYSIS: TOP 10 PRODUCTS BY REVENUE & PROFIT")
print("-" * 80)

top_products_revenue = df.groupby('Product Name').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Quantity': 'sum',
    'Order ID': 'count'
}).sort_values('Sales', ascending=False).head(10)
top_products_revenue.columns = ['Revenue', 'Profit', 'Units Sold', 'Orders']

print(f"\nTop 10 Products by Revenue:")
print(top_products_revenue)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

top_products_revenue['Revenue'].plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('Top 10 Products by Revenue', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Revenue ($)')

top_products_revenue['Profit'].plot(kind='barh', ax=axes[1], color='green')
axes[1].set_title('Top 10 Products by Profit', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Profit ($)')

plt.tight_layout()
plt.savefig('../reports/02_top_products.png', dpi=300, bbox_inches='tight')
print("✅ Saved: 02_top_products.png")
plt.close()

# ============================================================================
# ANALYSIS 3: CATEGORY-WISE PERFORMANCE
# ============================================================================
print("\n\n3️⃣ ANALYSIS: CATEGORY-WISE PERFORMANCE")
print("-" * 80)

category_performance = df.groupby('Category').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Order ID': 'count',
    'Quantity': 'sum'
}).reset_index()
category_performance.columns = ['Category', 'Revenue', 'Profit', 'Orders', 'Units']
category_performance['Profit_Margin_%'] = (category_performance['Profit'] / category_performance['Revenue'] * 100).round(2)

print(f"\nCategory Performance:")
print(category_performance)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

category_performance.set_index('Category')['Revenue'].plot(kind='bar', ax=axes[0, 0], color='skyblue')
axes[0, 0].set_title('Revenue by Category', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Revenue ($)')

category_performance.set_index('Category')['Profit'].plot(kind='bar', ax=axes[0, 1], color='green')
axes[0, 1].set_title('Profit by Category', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Profit ($)')

category_performance.set_index('Category')['Profit_Margin_%'].plot(kind='bar', ax=axes[1, 0], color='orange')
axes[1, 0].set_title('Profit Margin % by Category', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Profit Margin %')

category_performance.set_index('Category')['Orders'].plot(kind='bar', ax=axes[1, 1], color='purple')
axes[1, 1].set_title('Orders by Category', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Number of Orders')

plt.tight_layout()
plt.savefig('../reports/03_category_performance.png', dpi=300, bbox_inches='tight')
print("✅ Saved: 03_category_performance.png")
plt.close()

# ============================================================================
# ANALYSIS 4: NEW vs REPEAT CUSTOMERS
# ============================================================================
print("\n\n4️⃣ ANALYSIS: NEW vs REPEAT CUSTOMERS")
print("-" * 80)

customer_type_analysis = df.groupby('customer_type').agg({
    'Sales': ['sum', 'mean'],
    'Profit': ['sum', 'mean'],
    'Customer ID': 'nunique',
    'Order ID': 'count'
}).round(2)

print(f"\nNew vs Repeat Customer Analysis:")
print(customer_type_analysis)

new_pct = (df['customer_type'] == 'New').sum() / len(df) * 100
repeat_pct = (df['customer_type'] == 'Returning').sum() / len(df) * 100

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

customer_counts = df['customer_type'].value_counts()
axes[0].pie(customer_counts, labels=customer_counts.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('New vs Repeat Customer Distribution', fontsize=12, fontweight='bold')

customer_revenue = df.groupby('customer_type')['Sales'].sum()
axes[1].pie(customer_revenue, labels=customer_revenue.index, autopct='%1.1f%%', startangle=90)
axes[1].set_title('Revenue from New vs Repeat Customers', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('../reports/04_customer_type.png', dpi=300, bbox_inches='tight')
print("✅ Saved: 04_customer_type.png")
plt.close()

# ============================================================================
# ANALYSIS 5: HIGH-VALUE CUSTOMERS (PARETO ANALYSIS)
# ============================================================================
print("\n\n5️⃣ ANALYSIS: HIGH-VALUE CUSTOMERS (PARETO ANALYSIS)")
print("-" * 80)

customer_lifetime_value = df.groupby('Customer ID').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Order ID': 'count'
}).sort_values('Sales', ascending=False).reset_index()
customer_lifetime_value.columns = ['Customer ID', 'Total_Sales', 'Total_Profit', 'Num_Orders']

# Calculate cumulative percentage
total_revenue = customer_lifetime_value['Total_Sales'].sum()
customer_lifetime_value['Cumulative_Revenue'] = customer_lifetime_value['Total_Sales'].cumsum()
customer_lifetime_value['Cumulative_Pct'] = (customer_lifetime_value['Cumulative_Revenue'] / total_revenue * 100)

# Find 80% threshold
customers_for_80_pct = (customer_lifetime_value['Cumulative_Pct'] <= 80).sum()
pct_of_customers = (customers_for_80_pct / len(customer_lifetime_value) * 100)

print(f"\nPareto Analysis:")
print(f"Total Unique Customers: {len(customer_lifetime_value)}")
print(f"Customers for 80% Revenue: {customers_for_80_pct} ({pct_of_customers:.1f}%)")
print(f"Top 20% customers contribute to: {customer_lifetime_value['Cumulative_Pct'].iloc[int(len(customer_lifetime_value)*0.2)]:.1f}% of revenue")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(customer_lifetime_value['Cumulative_Pct'].values, linewidth=2, label='Cumulative Revenue %')
axes[0].axhline(y=80, color='r', linestyle='--', label='80% threshold')
axes[0].axvline(x=customers_for_80_pct, color='g', linestyle='--', label=f'Customers needed ({customers_for_80_pct})')
axes[0].set_title('Pareto Curve - Customer Contribution to Revenue', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Customer Rank')
axes[0].set_ylabel('Cumulative Revenue %')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

top_20_customers = customer_lifetime_value.head(int(len(customer_lifetime_value)*0.2))
axes[1].hist(top_20_customers['Num_Orders'], bins=20, color='steelblue', edgecolor='black')
axes[1].set_title('Order Frequency - Top 20% Customers', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Number of Orders')
axes[1].set_ylabel('Count')

plt.tight_layout()
plt.savefig('../reports/05_pareto_analysis.png', dpi=300, bbox_inches='tight')
print("✅ Saved: 05_pareto_analysis.png")
plt.close()

# ============================================================================
# ANALYSIS 6: CUSTOMER CHURN SIGNALS
# ============================================================================
print("\n\n6️⃣ ANALYSIS: CUSTOMER CHURN SIGNALS")
print("-" * 80)

# Customers with high purchase frequency but haven't ordered recently
df['days_since_order'] = (df['Order Date'].max() - df['Order Date']).dt.days

churn_risk = df.groupby('Customer ID').agg({
    'days_since_order': 'min',  # Most recent order
    'Order ID': 'count',
    'Sales': 'sum'
}).reset_index()
churn_risk.columns = ['Customer ID', 'Days_Since_Last_Order', 'Lifetime_Orders', 'Lifetime_Sales']

# Define churn: >180 days since last order
high_risk_churn = (churn_risk['Days_Since_Last_Order'] > 180).sum()
medium_risk_churn = ((churn_risk['Days_Since_Last_Order'] > 90) & (churn_risk['Days_Since_Last_Order'] <= 180)).sum()

print(f"\nChurn Risk Analysis:")
print(f"High Risk (>180 days): {high_risk_churn} customers ({high_risk_churn/len(churn_risk)*100:.1f}%)")
print(f"Medium Risk (90-180 days): {medium_risk_churn} customers ({medium_risk_churn/len(churn_risk)*100:.1f}%)")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(churn_risk['Days_Since_Last_Order'], bins=30, color='coral', edgecolor='black')
axes[0].axvline(x=180, color='r', linestyle='--', linewidth=2, label='High Risk (180 days)')
axes[0].axvline(x=90, color='orange', linestyle='--', linewidth=2, label='Medium Risk (90 days)')
axes[0].set_title('Days Since Last Order Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Days')
axes[0].set_ylabel('Number of Customers')
axes[0].legend()

churn_categories = pd.cut(churn_risk['Days_Since_Last_Order'], bins=[0, 90, 180, float('inf')], labels=['Low Risk', 'Medium Risk', 'High Risk'])
churn_counts = churn_categories.value_counts()
axes[1].pie(churn_counts, labels=churn_counts.index, autopct='%1.1f%%', startangle=90)
axes[1].set_title('Customer Churn Risk Distribution', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('../reports/06_churn_analysis.png', dpi=300, bbox_inches='tight')
print("✅ Saved: 06_churn_analysis.png")
plt.close()

# ============================================================================
# ANALYSIS 7: REGION-WISE REVENUE & PROFIT
# ============================================================================
print("\n\n7️⃣ ANALYSIS: REGION-WISE REVENUE & PROFIT")
print("-" * 80)

region_performance = df.groupby('Region').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Order ID': 'count',
    'Quantity': 'sum'
}).reset_index()
region_performance.columns = ['Region', 'Revenue', 'Profit', 'Orders', 'Units']
region_performance['Profit_Margin_%'] = (region_performance['Profit'] / region_performance['Revenue'] * 100).round(2)

print(f"\nRegion Performance:")
print(region_performance)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

region_performance.set_index('Region')['Revenue'].plot(kind='bar', ax=axes[0, 0], color='steelblue')
axes[0, 0].set_title('Revenue by Region', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Revenue ($)')

region_performance.set_index('Region')['Profit'].plot(kind='bar', ax=axes[0, 1], color='green')
axes[0, 1].set_title('Profit by Region', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Profit ($)')

region_performance.set_index('Region')['Profit_Margin_%'].plot(kind='bar', ax=axes[1, 0], color='orange')
axes[1, 0].set_title('Profit Margin % by Region', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Profit Margin %')

region_performance.set_index('Region')['Orders'].plot(kind='bar', ax=axes[1, 1], color='purple')
axes[1, 1].set_title('Orders by Region', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Number of Orders')

plt.tight_layout()
plt.savefig('../reports/07_region_analysis.png', dpi=300, bbox_inches='tight')
print("✅ Saved: 07_region_analysis.png")
plt.close()

# ============================================================================
# ANALYSIS 8: DISCOUNT IMPACT ANALYSIS
# ============================================================================
print("\n\n8️⃣ ANALYSIS: DISCOUNT IMPACT ON PROFITABILITY")
print("-" * 80)

# Create discount bins
df['discount_bin'] = pd.cut(df['Discount'], bins=[-0.01, 0, 0.1, 0.2, 0.3, 1], labels=['No Discount', '1-10%', '11-20%', '21-30%', '>30%'])

discount_impact = df.groupby('discount_bin', observed=True).agg({
    'Sales': ['sum', 'count'],
    'Profit': 'sum',
    'profit_margin': 'mean'
}).round(2)

print(f"\nDiscount Impact Analysis:")
print(discount_impact)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

discount_profit_margin = df.groupby('discount_bin', observed=True)['profit_margin'].mean()
discount_profit_margin.plot(kind='bar', ax=axes[0], color='coral')
axes[0].set_title('Profit Margin by Discount Level', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Profit Margin %')
axes[0].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45)

discount_volume = df.groupby('discount_bin', observed=True)['Order ID'].count()
discount_volume.plot(kind='bar', ax=axes[1], color='steelblue')
axes[1].set_title('Order Volume by Discount Level', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Number of Orders')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45)

plt.tight_layout()
plt.savefig('../reports/08_discount_impact.png', dpi=300, bbox_inches='tight')
print("✅ Saved: 08_discount_impact.png")
plt.close()

# ============================================================================
# CALCULATE KPIs
# ============================================================================
print("\n\n" + "="*80)
print("CALCULATING KEY PERFORMANCE INDICATORS (KPIs)")
print("="*80)

kpis = {}

# KPI 1: Total Revenue
kpis['Total Revenue'] = f"${df['Sales'].sum():,.2f}"

# KPI 2: Total Profit
kpis['Total Profit'] = f"${df['Profit'].sum():,.2f}"

# KPI 3: Overall Profit Margin
overall_margin = (df['Profit'].sum() / df['Sales'].sum() * 100)
kpis['Profit Margin (%)'] = f"{overall_margin:.2f}%"

# KPI 4: Average Order Value
aov = df['Sales'].mean()
kpis['Average Order Value'] = f"${aov:.2f}"

# KPI 5: Total Orders
kpis['Total Orders'] = f"{len(df):,}"

# KPI 6: Unique Customers
kpis['Unique Customers'] = f"{df['Customer ID'].nunique():,}"

# KPI 7: Revenue per Customer
rpc = df['Sales'].sum() / df['Customer ID'].nunique()
kpis['Revenue Per Customer'] = f"${rpc:.2f}"

# KPI 8: Repeat Customer Rate
repeat_rate = (df['customer_type'] == 'Returning').sum() / df['Customer ID'].nunique() * 100
kpis['Repeat Customer Rate (%)'] = f"{repeat_rate:.2f}%"

# KPI 9: Profit per Order
ppo = df['Profit'].sum() / len(df)
kpis['Profit Per Order'] = f"${ppo:.2f}"

# KPI 10: Discount Penetration
discount_rate = (df['Discount'] > 0).sum() / len(df) * 100
kpis['Discount Penetration (%)'] = f"{discount_rate:.2f}%"

# KPI 11: Average Delivery Days
avg_delivery = df['delivery_days'].mean() if 'delivery_days' in df.columns else 0
kpis['Average Delivery Days'] = f"{avg_delivery:.1f} days"

print(f"\n📊 KEY PERFORMANCE INDICATORS:")
print("-" * 80)
for key, value in kpis.items():
    print(f"{key:.<40} {value:>20}")

# Save KPIs to CSV
kpis_df = pd.DataFrame(list(kpis.items()), columns=['KPI', 'Value'])
kpis_df.to_csv('../data/processed/kpis.csv', index=False)
print("\n✅ KPIs saved to: data/processed/kpis.csv")

print("\n" + "="*80)
print("✨ PHASE 4 & 5 COMPLETE!")
print("="*80)
print("\nGenerated Files:")
print("  📊 8 Analysis visualizations in reports/")
print("  📈 kpis.csv with 11 KPIs in data/processed/")

✅ Data loaded: (9994, 39)
📅 Date range: 2014-01-03 00:00:00 to 2017-12-30 00:00:00

PHASE 4 & 5: EDA & KPI CALCULATIONS


1️⃣ ANALYSIS: MONTHLY & YEARLY REVENUE TRENDS
--------------------------------------------------------------------------------
Total Months: 48

Yearly Revenue Summary:
   Year      Revenue      Profit  Orders
0  2014  484247.4981  49543.9741    1993
1  2015  470532.5090  61618.6037    2102
2  2016  609205.5980  81795.1743    2587
3  2017  733215.2552  93439.2696    3312
✅ Saved: 01_revenue_trends.png


2️⃣ ANALYSIS: TOP 10 PRODUCTS BY REVENUE & PROFIT
--------------------------------------------------------------------------------

Top 10 Products by Revenue:
                                                      Revenue        Profit  \
Product Name                                                                  
Canon imageCLASS 2200 Advanced Copier               61599.824  2.519993e+04   
Fellowes PB500 Electric Punch Plastic Comb Bind...  27453.384  7.753039e+0

# MANDATORY ANALYSIS: SALES & REVENUE TRENDS